In [5]:
def installDeps():
    !pip install numpy scipy
    !pip install resampy tensorflow six
    !pip install youtube_dl
    !pip install ipywidgets
    !pip install pydub
    !pip install tqdm
    !pip install ffmpeg-python
    !apt-get install ffmpeg
    wget https://storage.googleapis.com/audioset/vggish_model.ckpt
#!python vggish_train_demo.py --num_batches 50 --train_vggish=False --checkpoint './vggish_model.ckpt'

from __future__ import print_function

from random import shuffle

import numpy as np
import tensorflow as tf

import vggish_input
import vggish_params
import vggish_slim
from pydub import AudioSegment
from audioUtils import readFolder

slim = tf.contrib.slim

In [6]:
def collectFilesForClass(c, max):
    dirs = readFolder('samples/%s' % (c))
    collected_files = []
    for d in dirs[:max]:
        files = readFolder('samples/%s/%s/out' % (c, d))

        for file in files:
            collected_files.append('samples/%s/%s/out/%s' % (c, d, file))
    return collected_files
            
def getSampleForFile(file):
    return AudioSegment.from_file(file).get_array_of_samples()

def getSamplesForFiles(files):
    sample = np.array([])
    
    sr = 44100  # Sampling rate.
    for file in files:
        
        sample = np.append(sample, getSampleForFile(file))
    return vggish_input.waveform_to_examples(sample, sr)   

def getData(files, arr):
    examples = getSamplesForFiles(files)
    labels = np.array([arr] * examples.shape[0])
    
    return (examples, labels)

def get_samples(shuf = True, num = None):
    #num_seconds = 5 * (len(files) * 2)
    #t = np.linspace(0, num_seconds, int(num_seconds * sr))  # Time axis.
    
    print('collecting laughter')
    files1 = collectFilesForClass('laughter', num)
    print('building data for files1')    
    x1, y1 = getData(files1, [1, 0])
    print('collecting not laughter')

    print('collecting notlaughter')
    files2 = collectFilesForClass('notlaughter', num)
    print('building data for files2')    
    x2, y2 = getData(files2, [0, 1])
    
    all_examples = np.concatenate((x1, x2))
    all_labels = np.concatenate((y1, y2))
    labeled_examples = list(zip(all_examples, all_labels))
    if shuf:
        shuffle(labeled_examples)

    # Separate and return the features and labels.
    features = [example for (example, _) in labeled_examples]
    labels = [label for (_, label) in labeled_examples]
    return (features, labels)

num = 2
(features, labels) = get_samples(shuf = False, num = num)
print(len(features))
np.save('features_%s.npy' % (num), features)
np.save('labels_%s.npy' % (num), labels)
# returns 3 classes, 5 sampels each, 96 numbers each

collecting laughter
building data for files1
collecting not laughter
collecting notlaughter
building data for files2
312


In [7]:
# Copyright 2017 The TensorFlow Authors All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

_NUM_CLASSES = 2


def train(get_examples, _NUM_BATCHES = 50):
    with tf.Graph().as_default(), tf.Session() as sess:
        pred = None
        # Define VGGish.
        embeddings = vggish_slim.define_vggish_slim(True) # Do we train VGG-ish?

        # Define a shallow classification model and associated training ops on top
        # of VGGish.
        with tf.variable_scope('mymodel'):
            # Add a fully connected layer with 100 units.
            num_units = 100
            fc = slim.fully_connected(embeddings, num_units)

            # Add a classifier layer at the end, consisting of parallel logistic
            # classifiers, one per class. This allows for multi-class tasks.
            logits = slim.fully_connected(
              fc, _NUM_CLASSES, activation_fn=None, scope='logits')
            pred = tf.sigmoid(logits, name='prediction')

            # Add training ops.
            with tf.variable_scope('train'):
                global_step = tf.Variable(
                    0, name='global_step', trainable=False,
                    collections=[tf.GraphKeys.GLOBAL_VARIABLES,
                                 tf.GraphKeys.GLOBAL_STEP])

            # Labels are assumed to be fed as a batch multi-hot vectors, with
            # a 1 in the position of each positive class label, and 0 elsewhere.
            labels = tf.placeholder(
                tf.float32, shape=(None, _NUM_CLASSES), name='labels')

            # Cross-entropy label loss.
            xent = tf.nn.sigmoid_cross_entropy_with_logits(
                logits=logits, labels=labels, name='xent')
            loss = tf.reduce_mean(xent, name='loss_op')
            tf.summary.scalar('loss', loss)

            # We use the same optimizer and hyperparameters as used to train VGGish.
            optimizer = tf.train.AdamOptimizer(
                learning_rate=vggish_params.LEARNING_RATE,
                epsilon=vggish_params.ADAM_EPSILON)
            optimizer.minimize(loss, global_step=global_step, name='train_op')

        # Initialize all variables in the model, and then load the pre-trained
        # VGGish checkpoint.
        sess.run(tf.global_variables_initializer())
        vggish_slim.load_vggish_slim_checkpoint(sess, './vggish_model.ckpt')

        # Locate all the tensors and ops we need for the training loop.
        features_tensor = sess.graph.get_tensor_by_name(
            vggish_params.INPUT_TENSOR_NAME)
        #for op in tf.get_default_graph().get_operations():
            #print(str(op.name))

        labels_tensor = sess.graph.get_tensor_by_name('mymodel/labels:0')
        #labels_tensor = sess.graph.get_tensor_by_name('mymodel/train/labels:0')    
        global_step_tensor = sess.graph.get_tensor_by_name(
            'mymodel/train/global_step:0')
        loss_tensor = sess.graph.get_tensor_by_name('mymodel/loss_op:0')
        train_op = sess.graph.get_operation_by_name('mymodel/train_op')

        # The training loop.
        for _ in range(_NUM_BATCHES):
          (features, labels) = get_examples(shuf=True)
          [num_steps, loss, _] = sess.run(
              [global_step_tensor, loss_tensor, train_op],
              feed_dict={features_tensor: features, labels_tensor: labels})
          print('Step %d: loss %g' % (num_steps, loss))
        saver = tf.train.Saver()
        saver.save(sess, './model/model_%s' %s (num))
        
        
        
        
        
        
        
        
        
        #pproc = vggish_postprocess.Postprocessor('./vggish_pca_params.npz')
        #print(vggish_params)
        # Define the model in inference mode, load the checkpoint, and
        # locate input and output tensors.
        #vggish_slim.define_vggish_slim(training=False)
        #vggish_slim.load_vggish_slim_checkpoint(sess, './vggish_model.ckpt')
        features_tensor = sess.graph.get_tensor_by_name(
            vggish_params.INPUT_TENSOR_NAME)
        embedding_tensor = sess.graph.get_tensor_by_name(
            vggish_params.OUTPUT_TENSOR_NAME)
        prediction=tf.argmax(logits,1)
        (features, labels) = get_examples(shuf=False)
        print(labels)
        #best = sess.run([prediction],feed_dict)
        # Run inference and postprocessing.
        embedding_batch = sess.run(pred,
                                     feed_dict={features_tensor: features})
        #print(embedding_batch.shape)
        #print(embedding_batch)
        #preds = sess.run(tf.nn.softmax(embedding_batch[0], 1))
        #print(preds)
        #preds=tf.argmax(embedding_batch,1)
        #print(preds.eval(feed_dict={features_tensor: features}))
        return embedding_batch 
    
def getSavedSamples(shuf = True):
    features = np.load('features_%s.npy' % (num))
    labels = np.load('labels_%s.npy' % (num))
    
    labeled_examples = list(zip(features, labels))
    if shuf:
        shuffle(labeled_examples)

    # Separate and return the features and labels.
    features = [example for (example, _) in labeled_examples]
    labels = [label for (_, label) in labeled_examples]
    return (features, labels)

preds = train(getSavedSamples, 50)

with tf.Graph().as_default(), tf.Session() as sess:
    print(preds)
    print(sess.run(tf.argmax(input=preds, axis=1)))

ValueError: The passed save_path is not a valid checkpoint: ./vggish_model.ckpt

In [ ]:
import vggish_postprocess

(features, labels) = _get_examples_batch2()
with tf.Graph().as_default(), tf.Session() as sess:
    pproc = vggish_postprocess.Postprocessor('./vggish_pca_params.npz')
    # Define the model in inference mode, load the checkpoint, and
    # locate input and output tensors.
    vggish_slim.define_vggish_slim(training=False)
    vggish_slim.load_vggish_slim_checkpoint(sess, './vggish_model.ckpt')
    features_tensor = sess.graph.get_tensor_by_name(
        vggish_params.INPUT_TENSOR_NAME)
    embedding_tensor = sess.graph.get_tensor_by_name(
        vggish_params.OUTPUT_TENSOR_NAME)

    # Run inference and postprocessing.
    [embedding_batch] = sess.run([embedding_tensor],
                                 feed_dict={features_tensor: features})
    print(embedding_batch)
    postprocessed_batch = pproc.postprocess(embedding_batch)
    print(postprocessed_batch)

    # Write the postprocessed embeddings as a SequenceExample, in a similar
    # format as the features released in AudioSet. Each row of the batch of
    # embeddings corresponds to roughly a second of audio (96 10ms frames), and
    # the rows are written as a sequence of bytes-valued features, where each
    # feature value contains the 128 bytes of the whitened quantized embedding.
    seq_example = tf.train.SequenceExample(
        feature_lists=tf.train.FeatureLists(
            feature_list={
                vggish_params.AUDIO_EMBEDDING_FEATURE_NAME:
                    tf.train.FeatureList(
                        feature=[
                            tf.train.Feature(
                                bytes_list=tf.train.BytesList(
                                    value=[embedding.tobytes()]))
                            for embedding in postprocessed_batch
                        ]
                    )
            }
        )
    )
    print(seq_example)

In [ ]:

files = [
    '%s/0.000000_5.000000.wav' % (directory),
    '%s/5.000000_10.000000.wav' % (directory),
    '%s/10.000000_15.000000.wav' % (directory),
    #'%s/15.000000_20.000000.wav' % (directory),
    #'%s/20.000000_25.000000.wav' % (directory),    
]
def _get_examples_batch2(shuf = True):
    """Returns a shuffled batch of examples of all audio classes.

    Note that this is just a toy function because this is a simple demo intended
    to illustrate how the training code might work.

    Returns:
    a tuple (features, labels) where features is a NumPy array of shape
    [batch_size, num_frames, num_bands] where the batch_size is variable and
    each row is a log mel spectrogram patch of shape [num_frames, num_bands]
    suitable for feeding VGGish, while labels is a NumPy array of shape
    [batch_size, num_classes] where each row is a multi-hot label vector that
    provides the labels for corresponding rows in features.
    """
    # Make a waveform for each class.
    num_seconds = 5 * (len(files) * 2)
    
    sr = 44100  # Sampling rate.
    t = np.linspace(0, num_seconds, int(num_seconds * sr))  # Time axis.
    
    # White noise.
    noise = np.random.normal(-1, 1, size=t.shape)

    sample = np.array([])
    for file in files:
        sample = np.append(sample, AudioSegment.from_file(file).get_array_of_samples())
    #sample = np.array(samples)
    
    #print(first_sample.shape)
    #sine_examples = []
    #for file in files:
    #    sine_examples.append(AudioSegment.from_file(files[0]).get_array_of_samples()[:96])

    #sine_examples = np.array(sine_examples)
    # Random sine wave.

    # Make examples of each signal and corresponding labels.
    # Sine is class index 0, Const class index 1, Noise class index 2.
    sine_examples = vggish_input.waveform_to_examples(sample, sr)
    
    
    
    #freq = np.random.uniform(100, 1000)
    #sine = np.sin(2 * np.pi * freq * t)
    #magnitude = np.random.uniform(-1, 1)
    #print(magnitude)
    #const = -1 * t
    # Random constant signal.
    #magnitude2 = np.random.uniform(-1, 1)
    #print(magnitude2)
    #const2 = 0 * t
    # White noise.
    noise = np.random.normal(-1, 1, size=t.shape)

    # Make examples of each signal and corresponding labels.
    # Sine is class index 0, Const class index 1, Noise class index 2.
    #sine_examples = vggish_input.waveform_to_examples(const, sr)
    sine_labels = np.array([[1, 0]] * sine_examples.shape[0])
    #sine_examples2 = vggish_input.waveform_to_examples(const2, sr)
    #sine_labels2 = np.array([[0, 1]] * sine_examples.shape[0])    
    #const_examples = vggish_input.waveform_to_examples(const, sr)
    #const_labels = np.array([[0, 1, 0]] * const_examples.shape[0])
    noise_examples = vggish_input.waveform_to_examples(noise, sr)
    noise_labels = np.array([[0, 1]] * noise_examples.shape[0])

    #print(sine_examples[0])
    #print('---')
    #print(sine_examples2[0])

    #print(noise_examples.shape)

    # Shuffle (example, label) pairs across all classes.
    all_examples = np.concatenate((sine_examples, noise_examples))
    all_labels = np.concatenate((sine_labels, noise_labels))
    labeled_examples = list(zip(all_examples, all_labels))
    if shuf:
        shuffle(labeled_examples)

    # Separate and return the features and labels.
    features = [example for (example, _) in labeled_examples]
    labels = [label for (_, label) in labeled_examples]
    return (features, labels)

(features, labels) = _get_examples_batch2()
print(len(features))
# returns 3 classes, 5 sampels each, 96 numbers each